# Database set up

- get user urls √
- test that one works
- test mongo
- setup workflow
    - batch the users
    

In [36]:
from bs4 import BeautifulSoup as bs
import urllib.request
import requests
import time
import re
import pandas as pd
import pymongo
from pymongo import MongoClient

## Setup the mongo Database

In [57]:
client = MongoClient()

In [58]:
db = client.book_ratings
book_collection = db.book_collection

In [ ]:
# need to run these two blocks in the function

#results = book_collection.insert_many()

In [ ]:
#this is to check

#results.inserted_id


## Read in the users urls

In [13]:
user_ids_2 = pd.read_csv('csv_files/user_base_urls.csv' , encoding='utf-8', header=None)

In [31]:
user_ids_2.columns

Int64Index([0], dtype='int64')

In [33]:
users_id_list= user_ids_2[0].tolist()

In [35]:
len(users_id_list)

3000

In [37]:
users_id_list[0]

'/review/list/4166936-cora-tea-party-princess'

## Function to get the rating and book ids from the users' ratings page
not getting the text comment, look at goodreads_data_1 for the code

In [71]:
def get_book_ratings(user_id):
    books = []
    ratings = []
    for i in range(120):
        review = f'https://www.goodreads.com{user_id}?sort=rating&view=reviews&page={i+1}'
        
        time.sleep(.6)
        
        r = urllib.request.urlopen(review)
        
        soup_r = bs(r, 'html.parser')
        
        x = soup_r.find_all('tr', {'class': 'bookalike review'})

        ratings.append([x[i].find('td', {'class':'field avg_rating'}).text[14:-1] for i in range(len(x))])

        books.append([x[i].a['href'] for i in range(len(x))])

        user_reviews= {b: r for b, r in zip(books[0], ratings[0])}
        
        user_reviews['user'] = user_id
        
        # take out print statement
#         if len(x) != 0:
#             print([x[i].a['href'] for i in range(len(x))])
        if len(x) == 0:
            break
    return  user_reviews

In [62]:
# to loop through user urls and get books and ratings
def books_for_users(user_url_list):
    info_users = []
    for usr in user_url_list:
        #print(reviews_url)
        info_users.append(get_book_ratings(usr))
    for j in info_users:
        for i in list(j.keys()):
            if i.find('.') !=-1:
                del info_users[info_users.index(j)][i]
    results = book_collection.insert_many(info_users).inserted_ids
    #results.inserted_ids
    return info_users

In [63]:
a = books_for_users(users_id_list[:2])
a

[{'/book/show/5985499-howl-s-moving-castle': '4.30',
  '/book/show/19105586-full-steam-ahead': '3.62',
  '/book/show/42776830-the-year-after-you': '4.90',
  '/book/show/18884761-a-girl-called-malice': '4.40',
  '/book/show/19440693-rebel-bride': '3.48',
  '/book/show/19539419-black-widow-volume-1': '4.05',
  '/book/show/24926015-simon-vs-the-homo-sapiens-agenda': '4.33',
  '/book/show/19883469-rose-bride': '3.65',
  '/book/show/25443389-extraordinary-means': '3.97',
  '/book/show/18778603-i-see-london': '3.87',
  '/book/show/18760207-white-hart': '3.72',
  '/book/show/40495957-heartstopper': '4.70',
  '/book/show/3865859-imposible': '3.97',
  '/book/show/20596747-storm-in-a-b-cup---a-breast-cancer-tale': '4.17',
  '/book/show/18665259-picture-perfect': '4.12',
  '/book/show/18664167-mind-games': '3.93',
  '/book/show/18657686-war-girls': '3.81',
  '/book/show/18522351-the-colour-of-magic': '3.99',
  '/book/show/40126081-fated': '3.93',
  '/book/show/20654100-the-castle': '3.89',
  '/bo

In [64]:
b = books_for_users(users_id_list[2:20])
b

[{'user': '/review/list/763271-nancy',
  '_id': ObjectId('5c5324a60db87462115e6870')},
 {'user': '/review/list/4859493-pepperp0t',
  '_id': ObjectId('5c5324a60db87462115e6871')},
 {'/book/show/25211206-across-the-pond': '4.45',
  '/book/show/21912494-rolling-thunder': '4.01',
  '/book/show/17857648-the-book-thief': '4.37',
  '/book/show/18619684-the-time-traveler-s-wife': '3.96',
  '/book/show/2767052-the-hunger-games': '4.33',
  'user': '/review/list/33931822-destiny-brown',
  '_id': ObjectId('5c5324a60db87462115e6872')},
 {'/book/show/6527051-krishna': '4.31',
  '/book/show/25912157-the-monk-who-sold-his-ferrari': '3.81',
  '/book/show/27036300-box-3014': '5.00',
  '/book/show/25800088-ashoka': '4.25',
  '/book/show/33959618-no-mans-land': '5.00',
  '/book/show/30739706-make-your-husband-your-boyfriend-again': '4.11',
  '/book/show/18615070-success-2020': '4.60',
  '/book/show/18264279-nagayan-collection': '4.26',
  '/book/show/33512151-teenage-diaries-the-days-that-were': '4.19',
  

In [65]:
c = books_for_users(users_id_list[20:50])
c

[{'/book/show/24275704-the-wake': '4.37',
  '/book/show/17899948-rebecca': '4.22',
  '/book/show/6609203-the-hound-of-the-baskervilles': '4.14',
  'user': '/review/list/34101827-alicia-ehrhardt',
  '_id': ObjectId('5c5332f60db87462115e6882')},
 {'/book/show/17337253-the-rainbow-virus': '3.87',
  '/book/show/31364983-shield-of-drani': '4.01',
  '/book/show/34096799-in-the-space-of-an-atom': '4.00',
  '/book/show/6944712-under-the-amoral-bridge': '3.69',
  '/book/show/32826681-don-t-look-back': '4.20',
  '/book/show/32283502-murder-in-vail': '4.29',
  '/book/show/31286362-the-rainbow-virus': '3.87',
  'user': '/review/list/17488666-dennis-meredith',
  '_id': ObjectId('5c5332f60db87462115e6883')},
 {'/book/show/9832272-the-haunting-of-hill-house': '3.87',
  '/book/show/11740839-christine': '3.75',
  '/book/show/31123849-savages': '4.05',
  '/book/show/30831912-norse-mythology': '4.10',
  '/book/show/7131611-lightning': '4.07',
  '/book/show/17888393-the-turtle-boy': '3.59',
  '/book/show/

In [66]:
d = books_for_users(users_id_list[50:70])
d

[{'/book/show/2713290-once-a-marshal': '4.00',
  '/book/show/2688185-once-hell-freezes-over': '4.15',
  '/book/show/2936415-where-men-win-glory': '4.05',
  '/book/show/35021921-the-punk-and-the-professor': '4.83',
  '/book/show/2579284-six-days-of-the-condor': '4.09',
  '/book/show/34479611-revenger': '4.50',
  '/book/show/33863683-natalie-s-dilemma': '4.86',
  '/book/show/32738388-rattlesnake-convention': '3.94',
  '/book/show/31868411-nose-candy-racketeer': '4.64',
  '/book/show/2713289-once-more-with-a-44': '4.30',
  'user': '/review/list/60599035-p-lundburg',
  '_id': ObjectId('5c533b5e0db87462115e68a0')},
 {'/book/show/30201060-the-way-of-happiness': '5.00',
  '/book/show/22548095-a-heart-of-dreams': '5.00',
  '/book/show/26164019-the-light-of-the-soul': '5.00',
  '/book/show/24282781-the-serpent-beguiled-eve': '4.45',
  '/book/show/24945025-beyond-the-skyline': '4.65',
  '/book/show/25016249-the-children-of-the-maize': '4.14',
  '/book/show/24624670-the-children-of-the-maize': '4

In [67]:
e = books_for_users(users_id_list[70:100])
e

[{'/book/show/24550342-siren-s-reckoning': '4.28',
  '/book/show/23873499-power-play': '4.31',
  '/book/show/23705801-the-cartel-bright-star': '4.00',
  '/book/show/24015781-a-life-away': '4.25',
  '/book/show/24152012-belonging': '4.18',
  '/book/show/25904465-the-braille-club': '4.26',
  '/book/show/23563038-rise-of-the-phoenix': '4.75',
  '/book/show/23562925-transgressions': '4.47',
  '/book/show/24468673-the-lightning-struck-heart': '4.48',
  '/book/show/23545333-surrendering': '3.94',
  '/book/show/24495549-chase-and-capture': '3.83',
  '/book/show/42377268-obsessed-4': '4.30',
  '/book/show/23520109-hontas': '4.27',
  '/book/show/24569417-the-united-states-of-africa': '3.00',
  '/book/show/24736408-feel-his-heat': '3.85',
  '/book/show/23470839-the-flesh-cartel-season-5': '4.44',
  '/book/show/24932334-feels-like-home': '3.82',
  '/book/show/23283023-fat-girlfriend': '4.62',
  '/book/show/24677224-in-the-shadow-of-angels': '4.26',
  '/book/show/23557129-happily-evan-after': '4.3

In [70]:
f = books_for_users(users_id_list[100:125])
f

[{'/book/show/29860211-smooth-talker': '4.08',
  '/book/show/28103960-the-alcohol-murders': '3.51',
  '/book/show/27424214-the-dark-strangler': '3.31',
  '/book/show/17445027-a-ton-of-gold': '4.22',
  '/book/show/26243942-hell-s-angels-biker-wars': '3.50',
  '/book/show/26170260-dual-impressions': '4.43',
  '/book/show/26367238-cinderella-teaser': '3.52',
  '/book/show/25928302-and-then-i-died': '4.29',
  '/book/show/25516979-justice-shall-be-served': '4.88',
  '/book/show/25480810-chiron-review---issue-97-fall-2014': '4.83',
  '/book/show/25522901-marc-l-pine': '3.37',
  '/book/show/25187259-marc-l-pine': '3.37',
  '/book/show/6356856-fools-rush-in': '4.53',
  '/book/show/23952823-all-due-respect-issue-4': '4.60',
  '/book/show/23481603-400-things-cops-know': '4.01',
  '/book/show/22920896-tales-from-dystopia': '5.00',
  '/book/show/16194232-the-stranger-beside-me': '4.14',
  '/book/show/22551730-dead-wake': '4.08',
  '/book/show/22469079-executor-rising': '3.82',
  '/book/show/623494

In [72]:
g = books_for_users(users_id_list[125:150])
g

[{'/book/show/38657219-tainted-reasoning': '4.65',
  '/book/show/35340409-controlling-heritage': '4.41',
  '/book/show/34977557-controlling-darkness': '4.61',
  '/book/show/38468034-controlling-disgrace': '4.66',
  '/book/show/36202634-fighting-the-lies': '4.58',
  '/book/show/37510939-fallen-for-shame': '4.49',
  '/book/show/35216609-the-touch-of-snow': '4.01',
  '/book/show/33279920-divided-control': '4.55',
  '/book/show/32786943-surrendered-control': '3.93',
  '/book/show/41006562-back-to-you': '4.80',
  '/book/show/8273406-suddenly-you': '3.98',
  '/book/show/40708766-unbroken': '4.79',
  '/book/show/41435276-beauty-s-war': '4.32',
  '/book/show/40374484-beauty-s-war': '4.32',
  '/book/show/40036258-unbreakable': '4.73',
  '/book/show/39962701-asphalt-cowboy-s-girl': '4.51',
  'user': '/review/list/51044388-j-l-leslie',
  '_id': ObjectId('5c5359c60db87462115e68eb')},
 {'/book/show/17167166-crown-of-midnight': '4.47',
  '/book/show/17910124-the-emperor-s-blades': '4.14',
  '/book/s

In [73]:
h = books_for_users(users_id_list[150:175])
h

[{'/book/show/19231295-machine-learning-with-r': '4.22',
  '/book/show/7913305-the-immortals-of-meluha': '4.05',
  '/book/show/18657648-doctor-who': '4.02',
  '/book/show/36239658-a-wasted-hour': '3.85',
  '/book/show/42603101-pieces-of-emeralds': '4.60',
  '/book/show/41561433-there-s-a-giant-trapdoor-spider-under-your-bed': '2.92',
  '/book/show/41642120-the-farewell-gift': '3.62',
  '/book/show/40670092-the-earth-crisis': '3.00',
  '/book/show/17707488-the-kill-list': '3.76',
  '/book/show/33541512-assassin-s-creed': '3.73',
  '/book/show/18224123-shooting-the-moon-and-other-stories': '4.00',
  '/book/show/24145970-think-and-grow-rich': '4.17',
  '/book/show/6969361-2-states': '3.42',
  'user': '/review/list/7650924-varun',
  '_id': ObjectId('5c535de30db87462115e6904')},
 {'user': '/review/list/23959375-k-y-l-a',
  '_id': ObjectId('5c535de30db87462115e6905')},
 {'/book/show/17048117-the-wakeful-world': '4.29',
  '/book/show/17131869-saga-vol-2': '4.55',
  '/book/show/17931235-balloo

In [74]:
i = books_for_users(users_id_list[175:200])
i

[{'/book/show/36672019-the-perils-of-being-moderately-famous': '3.40',
  '/book/show/35381714-calling-sehmat-a-novel': '3.89',
  '/book/show/35093953-loyal-stalkers': '4.31',
  '/book/show/28602817-the-girl-who-ate-books': '3.76',
  '/book/show/25870749-mrs-funnybones': '3.51',
  '/book/show/23658337-the-amazing-racist': '4.13',
  '/book/show/22014340-path-of-the-swan': '4.44',
  '/book/show/20642495-tinkle-digest-no-99': '4.04',
  '/book/show/17854648': '4.55',
  '/book/show/17706566': '4.59',
  'user': '/review/list/34423094-sucheta',
  '_id': ObjectId('5c5362b00db87462115e691d')},
 {'/book/show/15701962-odd-interlude-1': '4.11',
  '/book/show/15790417-triangulation': '4.67',
  '/book/show/38363799-dracul': '4.05',
  '/book/show/18516977-naked-in-the-light': '5.00',
  '/book/show/18517000-zombimboz': '5.00',
  '/book/show/35777443-triangulation': '4.70',
  '/book/show/33154920-the-fourth-monkey': '4.15',
  '/book/show/13154933-odd-apocalypse': '3.95',
  '/book/show/7260188-mockingjay

In [75]:
j = books_for_users(users_id_list[200:225])
j

[{'/book/show/31704952-the-sand-dweller': '4.52',
  '/book/show/30107600-wreath-and-other-stories': '4.35',
  '/book/show/19351833-kiterunner-w-myreadinglab': '4.28',
  '/book/show/32076440-the-devil-s-prayer': '3.98',
  '/book/show/11447065-the-quest': '4.09',
  '/book/show/23443453-the-big-bang-and-lines-of-space': '4.85',
  '/book/show/17823246-lines-of-space': '4.32',
  '/book/show/25812847-the-silk-roads': '4.25',
  '/book/show/28257707-the-subtle-art-of-not-giving-a-f-ck': '3.99',
  '/book/show/40773359-turning-the-tide-on-plastic': '4.16',
  '/book/show/32283133-origin': '3.85',
  '/book/show/29849037-god-s-wolf': '3.86',
  '/book/show/11781092-the-great-commanders-of-the-medieval-world-454-1582': '3.57',
  '/book/show/10079597-coal-gasification-and-its-applications': '4.00',
  '/book/show/28815475-genghis-khan-and-the-quest-for-god': '4.05',
  '/book/show/29503098-after-swissair': '5.00',
  '/book/show/28821802-home': '4.00',
  '/book/show/25327825-the-author-s-guide-to-working

In [76]:
k = books_for_users(users_id_list[225:250])
k

[{'/book/show/43494119-lies-my-teacher-told-me-for-young-readers': '4.75',
  '/book/show/36114332-don-t-let-go': '4.07',
  '/book/show/33133516-grounded': '4.25',
  '/book/show/32048650-dark': '3.91',
  '/book/show/35021880-only-the-few': '3.97',
  '/book/show/35890044-the-benefits-of-being-an-octopus': '4.52',
  '/book/show/36037331-forgetting': '4.00',
  '/book/show/19075922-hms-surprise-aubrey-maturin-series-book-3': '4.42',
  '/book/show/30002998-two-by-two': '3.91',
  '/book/show/23633448-the-letter-for-the-king': '4.15',
  '/book/show/34687941-cruising-alaska-on-a-budget': '4.27',
  '/book/show/25743752-my-name-is-leon': '4.02',
  '/book/show/25218773-bipolar-1-disorder': '4.27',
  '/book/show/6238661-rain-gods': '4.00',
  '/book/show/23592235-am-i-normal-yet': '4.27',
  '/book/show/8714383-flip': '3.53',
  '/book/show/23252527-toys-in-the-attic': '4.50',
  '/book/show/36472944-necrotic-city': '4.41',
  '/book/show/9644151-ashfall': '3.97',
  '/book/show/29744377-the-secret-diary

In [77]:
l = books_for_users(users_id_list[250:275])
l

[{'/book/show/34763922-carapace': '4.47',
  '/book/show/34670289-spider-gwen-vol-4': '3.67',
  '/book/show/3173503-ultra-fuckers': '3.95',
  '/book/show/17837762-uzumaki': '4.37',
  '/book/show/11166889-the-funhouse': '3.60',
  'user': '/review/list/56087861-casey-bartsch',
  '_id': ObjectId('5c5383cc0db87462115e6968')},
 {'/book/show/37793868-a-comprehensive-guide-to-digital-marketing-strategy': '4.25',
  '/book/show/20318407-marketing-as-a-business-system': '4.75',
  '/book/show/37793006-a-comprehensive-guide-to-digital-marketing-strategy': '4.00',
  '/book/show/21899722-touchdowns': '4.17',
  '/book/show/21899735-free-throws': '5.00',
  '/book/show/20404402-touchdowns': '4.17',
  '/book/show/25725068-minecraft': '4.34',
  'user': '/review/list/26872239-neil-kokemuller',
  '_id': ObjectId('5c5383cc0db87462115e6969')},
 {'/book/show/40400267-stranger-things': '4.57',
  '/book/show/36609155-robin': '4.20',
  '/book/show/29430466-labyrinth': '4.77',
  '/book/show/23846161-ghostbusters':

In [78]:
m = books_for_users(users_id_list[275:300])
m

[{'/book/show/35702821-six-nights-of-sin-the-complete-series': '4.48',
  '/book/show/33963498-hangman-s-army': '4.54',
  '/book/show/28532742-rise-of-the-river-man': '4.48',
  '/book/show/26808273-rise-of-the-river-man': '4.48',
  '/book/show/24984170-secrets-in-blood': '4.42',
  '/book/show/24237009-escape': '4.04',
  '/book/show/2767052-the-hunger-games': '4.33',
  '/book/show/26152442-pathosis': '4.14',
  '/book/show/21427834-damaged-like-us': '4.01',
  '/book/show/34390325-escape': '4.04',
  'user': '/review/list/38543082-l-s-o-dea',
  '_id': ObjectId('5c5389090db87462115e6981')},
 {'/book/show/5598113-vision-in-white': '4.11',
  '/book/show/7933292-heaven-is-for-real': '4.01',
  '/book/show/7989800-christmas-eve-at-friday-harbor': '3.75',
  '/book/show/7812659-safe-haven': '4.20',
  '/book/show/8520610-quiet': '4.06',
  'user': '/review/list/39453089-louise',
  '_id': ObjectId('5c5389090db87462115e6982')},
 {'/book/show/2767052-the-hunger-games': '4.33',
  '/book/show/36146358-sur

In [79]:
n = books_for_users(users_id_list[300:325])
n

[{'/book/show/11052224-the-lost-world': '3.92',
  '/book/show/10080974-david-copperfield': '3.98',
  '/book/show/15845322-the-picture-of-dorian-gray': '4.07',
  '/book/show/18681018-the-divine-comedy-of-dante-alighieri-the-inferno': '3.99',
  '/book/show/15848107-the-strange-case-of-dr-jekyll-and-mr-hyde': '3.80',
  '/book/show/6239806-oliver-twist': '3.86',
  '/book/show/17924421-tales-of-terror-and-mystery': '4.17',
  '/book/show/17261414-grimm-s-fairy-stories': '4.10',
  '/book/show/18615474-heart-of-darkness': '3.42',
  '/book/show/20634671-septimus': '3.85',
  '/book/show/22400870-similes-dictionary': '4.29',
  '/book/show/21137594-complete-works-of-confucius': '4.04',
  '/book/show/15756620-dracula': '3.99',
  '/book/show/28541538-slave-narrative-six-pack-5': '4.56',
  '/book/show/11058727-paradise-lost': '3.81',
  '/book/show/33670466-paperbacks-from-hell': '4.32',
  '/book/show/37917393-the-wicked-ones': '4.31',
  '/book/show/39931828-elevation': '3.74',
  'user': '/review/list

In [ ]:
o = books_for_users(users_id_list[325:350])
o

In [ ]:
q = books_for_users(users_id_list[350:375])
q

In [ ]:
r = books_for_users(users_id_list[375:400])
r

In [ ]:
s = books_for_users(users_id_list[400:425])
s

In [ ]:
t = books_for_users(users_id_list[425:450])
t

In [ ]:
u = books_for_users(users_id_list[450:475])
u

In [ ]:
v = books_for_users(users_id_list[475:500])
v

In [ ]:
w = books_for_users(users_id_list[500:525])
w

In [ ]:
x = books_for_users(users_id_list[525:550])
x

In [ ]:
y = books_for_users(users_id_list[550:575])
y

In [ ]:
z = books_for_users(users_id_list[575:600])
z

In [ ]:
aa = books_for_users(users_id_list[600:625])
aa

In [ ]:
bb = books_for_users(users_id_list[625:650])
bb

In [ ]:
cc = books_for_users(users_id_list[650:675])
cc

In [ ]:
dd = books_for_users(users_id_list[675:700])
dd

In [ ]:
ee = books_for_users(users_id_list[700:725])
ee

In [ ]:
ff = books_for_users(users_id_list[725:750])
ff

In [ ]:
gg = books_for_users(users_id_list[750:775])
gg

In [ ]:
hh = books_for_users(users_id_list[775:800])
hh

In [ ]:
ii = books_for_users(users_id_list[800:825])
ii

In [ ]:
jj = books_for_users(users_id_list[825:850])
jj

In [ ]:
kk = books_for_users(users_id_list[850:875])
kk

In [ ]:
ll = books_for_users(users_id_list[875:900])
ll

In [ ]:
mm = books_for_users(users_id_list[900:925])
mm

In [ ]:
nn = books_for_users(users_id_list[925:950])
nn

In [ ]:
oo = books_for_users(users_id_list[950:975])
oo

In [ ]:
pp = books_for_users(users_id_list[975:1000])
pp

In [ ]:
qq = books_for_users(users_id_list[1000:1025])
qq

In [ ]:
rr = books_for_users(users_id_list[1025:1050])
rr

In [ ]:
ss = books_for_users(users_id_list[1050:1075])
ss

In [ ]:
tt = books_for_users(users_id_list[1075:1100])
tt

In [ ]:
uu = books_for_users(users_id_list[1100:1125])
uu

In [ ]:
vv = books_for_users(users_id_list[1125:1150])
vv

In [ ]:
ww = books_for_users(users_id_list[1150:1175])
ww

In [ ]:
xx = books_for_users(users_id_list[1175:1200])
xx

In [ ]:
yy = books_for_users(users_id_list[1200:1225])
yy

In [ ]:
zz = books_for_users(users_id_list[1225:1250])
zz

In [ ]:
aaa = books_for_users(users_id_list[1250:1275])
aaa

In [ ]:
bbb = books_for_users(users_id_list[1275:1300])
bbb

In [ ]:
ccc = books_for_users(users_id_list[1300:1325])
ccc

In [ ]:
ddd = books_for_users(users_id_list[1325:1350])
ddd

In [ ]:
eee = books_for_users(users_id_list[1350:1375])
eee

In [ ]:
fff = books_for_users(users_id_list[1375:1400])
fff

In [ ]:
ggg = books_for_users(users_id_list[1400:1425])
ggg

In [ ]:
hhh = books_for_users(users_id_list[1425:1450])
hhh

In [ ]:
iii = books_for_users(users_id_list[1450:1475])
iii

In [ ]:
jjj = books_for_users(users_id_list[1475:1500])
jjj

In [ ]:
kkk = books_for_users(users_id_list[1500:1525])
kkk

In [ ]:
lll = books_for_users(users_id_list[1525:1550])
lll

In [ ]:
mmm = books_for_users(users_id_list[1550:1575])
mmm

In [ ]:
nnn = books_for_users(users_id_list[1575:1600])
nnn

In [ ]:
ooo = books_for_users(users_id_list[1600:1625])
ooo

In [ ]:
ppp = books_for_users(users_id_list[1625:1650])
ppp

In [ ]:
qqq = books_for_users(users_id_list[1650:1675])
qqq

In [ ]:
rrr = books_for_users(users_id_list[1675:1700])
rrr

In [ ]:
sss = books_for_users(users_id_list[1700:1725])
sss

In [ ]:
ttt = books_for_users(users_id_list[1725:1750])
ttt

In [ ]:
uuu = books_for_users(users_id_list[1750:1775])
uuu

In [ ]:
vvv = books_for_users(users_id_list[1775:1800])
vvv

In [ ]:
www = books_for_users(users_id_list[1800:1825])
www

In [ ]:
xxx = books_for_users(users_id_list[1825:1850])
xxx

In [ ]:
yyy = books_for_users(users_id_list[1850:1875])
yyy

In [ ]:
zzz = books_for_users(users_id_list[1875:1900])
zzz

In [ ]:
aaaa = books_for_users(users_id_list[1900:1925])
aaaa

In [ ]:
bbbb = books_for_users(users_id_list[1925:1950])
bbbb

In [ ]:
cccc = books_for_users(users_id_list[1950:1975])
cccc

In [ ]:
dddd = books_for_users(users_id_list[1975:2000])
dddd

In [ ]:
eeee = books_for_users(users_id_list[2000:2025])
eeee

In [ ]:
ffff = books_for_users(users_id_list[2025:2050])
ffff

In [ ]:
gggg = books_for_users(users_id_list[2075:2100])
gggg

In [ ]:
hhhh = books_for_users(users_id_list[2100:2125])
hhhh

In [ ]:
iiii = books_for_users(users_id_list[2125:2150])
iiii

In [ ]:
jjjj = books_for_users(users_id_list[2150:2175])
jjjj

In [ ]:
kkkk = books_for_users(users_id_list[2175:2200])
kkkk

In [ ]:
llll = books_for_users(users_id_list[2200:2225])
llll

In [ ]:
mmmm = books_for_users(users_id_list[2225:2250])
mmmm

In [ ]:
nnnn = books_for_users(users_id_list[2250:2275])
nnnn

In [ ]:
oooo = books_for_users(users_id_list[2275:2300])
oooo

In [ ]:
pppp = books_for_users(users_id_list[2300:2325])
pppp

In [ ]:
qqqq = books_for_users(users_id_list[2325:2350])
qqqq

In [ ]:
rrrr = books_for_users(users_id_list[2350:2375])
rrrr

In [ ]:
ssss = books_for_users(users_id_list[2375:2400])
ssss

In [ ]:
tttt = books_for_users(users_id_list[2400:2425])
tttt

In [ ]:
uuuu = books_for_users(users_id_list[2425:2450])
uuuu

In [ ]:
vvvv = books_for_users(users_id_list[2450:2475])
vvvv

In [ ]:
wwww = books_for_users(users_id_list[2475:2500])
wwww

In [ ]:
xxxx = books_for_users(users_id_list[2500:2525])
xxxx

In [ ]:
yyyy = books_for_users(users_id_list[2525:2550])
yyyy

In [ ]:
zzzz = books_for_users(users_id_list[2550:2575])
zzzz

In [ ]:
aaaaa = books_for_users(users_id_list[2575:2600])
aaaaa

In [ ]:
bbbbb = books_for_users(users_id_list[2600:2625])
bbbbb

In [ ]:
ccccc = books_for_users(users_id_list[2625:2650])
ccccc

In [ ]:
ddddd = books_for_users(users_id_list[2650:2675])
ddddd

In [ ]:
eeeee = books_for_users(users_id_list[2675:2700])
eeeee

In [ ]:
fffff = books_for_users(users_id_list[2700:2725])
fffff

In [ ]:
ggggg = books_for_users(users_id_list[2725:2750])
ggggg

In [ ]:
hhhhh = books_for_users(users_id_list[2750:2775])
hhhhh

In [ ]:
iiiii = books_for_users(users_id_list[2775:2800])
iiiii

In [ ]:
jjjjj = books_for_users(users_id_list[2800:2825])
jjjjj

In [ ]:
kkkkk = books_for_users(users_id_list[2825:2850])
kkkkk

In [ ]:
lllll = books_for_users(users_id_list[2850:2875])
lllll

In [ ]:
mmmmm = books_for_users(users_id_list[2875:2900])
mmmmm

In [ ]:
nnnnn = books_for_users(users_id_list[2900:2925])
nnnnn

In [ ]:
ooooo = books_for_users(users_id_list[2925:2950])
ooooo

In [ ]:
ppppp = books_for_users(users_id_list[2950:2975])
ppppp

In [ ]:
qqqqq = books_for_users(users_id_list[2975:3000])
qqqqq

# Break of the user ids into smaller lists

In [51]:
users_id_list[:2]

['/review/list/4166936-cora-tea-party-princess',
 '/review/list/14895890-rebecca-not-reviewing-at-the-moment']

In [54]:
users_id_list[2:20]

['/review/list/763271-nancy',
 '/review/list/4859493-pepperp0t',
 '/review/list/33931822-destiny-brown',
 '/review/list/49901449-rishit',
 '/review/list/16273240-april-wood',
 '/review/list/34170124-t-r-robinson',
 '/review/list/23016460-amber',
 '/review/list/6111245-rose-desert-rose',
 '/review/list/823548-p-i',
 '/review/list/31024389-john-autero',
 '/review/list/4174124-jola',
 '/review/list/23216850-ashok',
 '/review/list/18721584-raevyn-lucia-i-m-in-it-for-the-books',
 '/review/list/44430291-saradia-chatterjee',
 '/review/list/23423339-groovy-lee',
 '/review/list/34588624-tonya-coffey',
 '/review/list/33182504-prashant-pinge',
 '/review/list/62757015-lgandt']

In [56]:
users_id_list[20:50]

['/review/list/34101827-alicia-ehrhardt',
 '/review/list/17488666-dennis-meredith',
 '/review/list/16112063-david-brian',
 '/review/list/20729688-this-coull-chick',
 '/review/list/30997061-cy-wyss',
 '/review/list/60551076-joshua-grant',
 '/review/list/22943325-maxine-booklover-catlady',
 '/review/list/72965725-mfonemana-uduak',
 '/review/list/29251934-sam-fury',
 '/review/list/16156990-sebanti',
 '/review/list/4969315-a-j-locke',
 '/review/list/39491854-audrey-rich',
 '/review/list/14163098-bernadette-donnelly',
 '/review/list/33806697-noorilhuda',
 '/review/list/6206576-crystal',
 '/review/list/65888567-mala-mukherjee',
 '/review/list/43339522-margaret-chadwick',
 '/review/list/24877080-eric-james-olson',
 '/review/list/19520403-jenn-reader-of-things',
 '/review/list/16602614-david-staniforth',
 '/review/list/5743998-david-laing',
 '/review/list/6003473-m-a-levi',
 '/review/list/65139180-prakash-sharma',
 '/review/list/17077792-romeight',
 '/review/list/23265470-joanie-chevalier',
 '